In [ ]:
from pathlib import Path

import numpy as np

from gamma_surfaces import get_gamma_surface, show_master_gamma_surface, show_gamma_surface_fit
from utilities import make_structure, get_lammps_parameters

In [ ]:
pot_base_dir = Path('.').resolve()
common_lammps_params = get_lammps_parameters(base_path=pot_base_dir)

In [ ]:
# For conversion from eV / Ang^2 --> J / m^2
UNIT_CONV = 16.02176565

In [ ]:
# Note: if examining any of the twist GB gamma surfaces, the energy landscape is shown on orthogonal axes (when in fact the axes are non-orthogonal)
gs = get_gamma_surface('s7-tlA', 'dft')
show_master_gamma_surface(gs, 'grain_boundary_energy')

In [ ]:
gs = get_gamma_surface('s13-tlA', 'eam')
show_master_gamma_surface(gs, 'grain_boundary_energy')

# Visualise gamma surface data

The code below uses JSON data files that are stored within this repository to visualise the fitted gamma surfaces.

In [ ]:
structure_code = 's7-tlA'
method = 'dft'

gs = get_gamma_surface(structure_code, method)
show_master_gamma_surface(gs, data_name='grain_boundary_energy')

In [ ]:
structure_code = 's13-tlA'
method = 'eam'

gs = get_gamma_surface(structure_code, method)
show_master_gamma_surface(gs, data_name='grain_boundary_energy')

In [ ]:
# Get the translation with the minimum fitted grain boundary energy:
min_shift = gs.get_minimum_fitted_shift('grain_boundary_energy')[0].shift
min_shift

In [ ]:
# Show quadratic fitting at a given shift:
show_gamma_surface_fit(gs, shift=min_shift, data_name='grain_boundary_energy')

In [ ]:
# Show quadratic fitting at a given shift:
show_gamma_surface_fit(gs, shift=[0.25, 0.5], data_name='grain_boundary_energy')

In [ ]:
# Get the fitted minimum expansion and grain boundary energy:
gs.fitted_data['grain_boundary_energy']['minimum'][
    np.argmin(gs.fitted_data['grain_boundary_energy']['minimum'][:, 1]), :
]

# Method of gamma surface generation and analysis

This section breaks down the steps used to:

- generate gamma surface input files (for LAMMPS)
- run the simulations
- collate the results
- perform quadratic fitting
- visualise the master gamma surface

> **Note**: you cannot run simulations on Binder!

In [ ]:
from subprocess import run, PIPE
import numpy as np

from atomistic.bicrystal import GammaSurface
from lammps_parse import write_lammps_inputs, read_lammps_output
from plotly import graph_objects

# Change this variable to the directory where LAMMPS gamma surface simulation input files should be generated:
LAMMPS_SIMS_DIR = Path('/path/to/simulation/directory')

# Change this to the LAMMPS executable name (note, you cannot run simulations on Binder!):
LAMMPS_EXECUTABLE = 'lmp_serial'

**Generate a bicrystal whose gamma surface is to be explored**

In [ ]:
structure_key = 's7-tlA'

In [ ]:
bicrystal = make_structure(f'{structure_key}-gb', method='eam', configuration='sized')
bicrystal.show(
    include={'points': ['atoms']},
    layout_args={'height': 800},
)

**Generate a corresponding bulk model**

In [ ]:
bulk = make_structure(f'{structure_key}-b', method='eam', configuration='sized')
bulk.show(
    include={'points': ['atoms']},
    layout_args={'height': 600},
)

**Generate a gamma surface from a grid**

In [ ]:
gamma_surface = GammaSurface.from_grid(bicrystal, grid=[12, 18], expansions=[-1, 0, 1, 2])

**Iterate over all coordinates in the gamma surface to generate simulation input files**

In [ ]:
pot_base_dir = Path('.').resolve()
GB_input_paths = []

common_lammps_params = get_lammps_parameters(base_path=pot_base_dir)

for i in gamma_surface.all_coordinates():
    
    # Make a directory for this sim:
    sim_path = LAMMPS_SIMS_DIR.joinpath(f'{structure_key}-gb', i.coordinate_fmt)
    sim_path.mkdir(parents=True)
    
    # Write simulation inputs:
    lammps_params_gb_i = {
        'supercell': i.structure.supercell,
        'atom_sites': i.structure.atoms.coords,
        'species': i.structure.atoms.labels['species'].unique_values,
        'species_idx': i.structure.atoms.labels['species'].values_idx,
        'dir_path': sim_path,
        **common_lammps_params,
    }
    GB_input_paths.append(write_lammps_inputs(**lammps_params_gb_i))
    
# Make a directory for the bulk sim:
sim_path = LAMMPS_SIMS_DIR.joinpath(f'{structure_key}-b')
sim_path.mkdir()

# Write bulk simulation inputs:
lammps_params_bulk = {
    'supercell': bulk.supercell,
    'atom_sites': bulk.atoms.coords,
    'species': bulk.atoms.labels['species'].unique_values,
    'species_idx': bulk.atoms.labels['species'].values_idx,
    'dir_path': sim_path,
    **common_lammps_params,
}
bulk_input_path = write_lammps_inputs(**lammps_params_bulk)

**Run simulations**

In [ ]:
for i in GB_input_paths + [bulk_input_path]:
    cmd = '{} < {}'.format(LAMMPS_EXECUTABLE, i.name)
    proc = run(cmd, shell=True, cwd=i.parent , stdout=PIPE, stderr=PIPE)

**Collate simulation outputs**

In [ ]:
simulated_gamma_surface_params = {
    'shifts': [],
    'expansions': [],
    'data': {
        'energy': [],
        'grain_boundary_energy': [],
    },
    'metadata': {},
}
# First get the bulk sim data:
lammps_out_bulk = read_lammps_output(dir_path=bulk_input_path.parent)
E_tot_bulk = lammps_out_bulk['final_energy'][-1]
simulated_gamma_surface_params['metadata'].update({
    'E_tot_bulk': E_tot_bulk,
    'grain_boundary_area': bicrystal.boundary_area,
    'num_atoms_bulk': bulk.num_atoms,
    'num_atoms_grain_boundary': bicrystal.num_atoms,
})

# Now iterate over GB sims:
for i in GB_input_paths:
        
    shift_str, exp_str = i.parent.name.split('__')
    shift = []
    for j in shift_str.split('_'):
        num, denom = j.split('.')
        shift.append(int(num) / int(denom))
        
    simulated_gamma_surface_params['shifts'].append(shift)
    simulated_gamma_surface_params['expansions'].append(float(exp_str))
    
    lammps_out_GB_i = read_lammps_output(dir_path=i.parent)    
    E_tot_GB_i = lammps_out_GB_i['final_energy'][-1]
    simulated_gamma_surface_params['data']['energy'].append(E_tot_GB_i)    
    
    E_GB_i = (1 / (2 * bicrystal.boundary_area)) * (
        E_tot_GB_i - (bicrystal.num_atoms / bulk.num_atoms) * E_tot_bulk
    ) * UNIT_CONV
    simulated_gamma_surface_params['data']['grain_boundary_energy'].append(E_GB_i)

In [ ]:
simulated_gamma_surface = GammaSurface(bicrystal, **simulated_gamma_surface_params)

**Plot a slice of the gamma surface**

In [ ]:
data_label = 'grain_boundary_energy'
plot_dat = simulated_gamma_surface.get_surface_plot_data(data_label, expansion=0, xy_as_grid=False)
grid_dat = simulated_gamma_surface.get_xy_plot_data()
graph_objects.FigureWidget(
    data=[
        {
            'type': 'contour',
            'colorscale': 'viridis',
            'colorbar': {
                'title': data_label,
            },
            **plot_dat,            
        },
        {
            **grid_dat,
            'mode': 'markers',
            'marker': {
                'size': 2,
            },
        },
    ],
    layout={
        'xaxis': {
            'scaleanchor': 'y',            
        },        
    }
)

**Fit at each shift to find the master gamma surface, and plot**

In [ ]:
simulated_gamma_surface.add_fit(data_label, 3)

In [ ]:
master_plot_data = simulated_gamma_surface.get_fitted_surface_plot_data(data_label, xy_as_grid=False)
graph_objects.FigureWidget(
    data=[
        {
            'type': 'contour',
            'colorscale': 'viridis',
            **master_plot_data,            
        },
        {
            **grid_dat,
            'mode': 'markers',
            'marker': {
                'size': 2,
            },
        },        
    ],
    layout={
        'xaxis': {
            'scaleanchor': 'y',
        }
    }
)

**Plot the fit at a given shift**

In [ ]:
fit_plot_dat = simulated_gamma_surface.get_fit_plot_data(data_label, [0, 0])
fig = graph_objects.FigureWidget(
    data=[
        {
            **fit_plot_dat['fitted_data'],
            'name': 'Fit',
        },
        {
            **fit_plot_dat['data'],
            'name': data_label,
        },
        {
            **fit_plot_dat['minimum'],
            'name': 'Fit min.',
        },
    ],
    layout={
        'xaxis': {
            'title': 'Expansion',
        },
        'yaxis': {
            'title': data_label,
        },
        'width': 500,
        'height': 400,
    }
)
fig

# Using wrapper functions in `gamma_surfaces.py`

The above workflow is wrapped up into a few wrapper functions in the python file `gamma_surfaces.py`.

In [ ]:
from gamma_surfaces import compute_master_gamma_surface

In [ ]:
structure_code = 's7-tlA'
gamma_surface = compute_master_gamma_surface(structure_code, LAMMPS_SIMS_DIR) # 5-10 mins run time for the STGBs (much longer for the twists)

In [ ]:
show_master_gamma_surface(gamma_surface, data_name='grain_boundary_energy')

# LAMMPS simulations to find grain boundary energies

Use the minimum-energy configurations (shift and expansion) from the EAM γ-surfaces to do full-atom relaxations:

In [ ]:
import copy

In [ ]:
common_lammps_params = copy.deepcopy(common_lammps_params)
common_lammps_params['atom_constraints']['fix_xy_idx'] = None

In [ ]:
structure_codes = [
    's7-tlA',
    's13-tlA',
    's19-tlA',
    's31-tlA',
    's7-tw',
    's13-tw',
    's19-tw',
    's7-tlB',    
]
E_GB_EAM = {}
for structure_code in structure_codes:    
    bicrystal = make_structure(
        f'{structure_code}-gb',
        method='eam',
        configuration='minimum_energy',
    )
    bulk = make_structure(
        f'{structure_code}-b',
        method='eam',
        configuration='minimum_energy'
    )
    
    sim_path_GB = LAMMPS_SIMS_DIR.joinpath('full_relaxations', f'{structure_code}-gb')
    sim_path_GB.mkdir(parents=True)
    
    # Write simulation inputs:
    lammps_params_gb = {
        'supercell': bicrystal.supercell,
        'atom_sites': bicrystal.atoms.coords,
        'species': bicrystal.atoms.labels['species'].unique_values,
        'species_idx': bicrystal.atoms.labels['species'].values_idx,
        'dir_path': sim_path_GB,
        **common_lammps_params,
    }
    input_path_GB = write_lammps_inputs(**lammps_params_gb)    
    
    sim_path_bulk = LAMMPS_SIMS_DIR.joinpath('full_relaxations', f'{structure_code}-b', )
    sim_path_bulk.mkdir(parents=True)
    
    # Write simulation inputs:
    lammps_params_bulk = {
        'supercell': bulk.supercell,
        'atom_sites': bulk.atoms.coords,
        'species': bulk.atoms.labels['species'].unique_values,
        'species_idx': bulk.atoms.labels['species'].values_idx,
        'dir_path': sim_path_bulk,
        **common_lammps_params,
    }
    input_path_bulk = write_lammps_inputs(**lammps_params_bulk)      
    
    for i in [input_path_GB, input_path_bulk]:
        cmd = '{} < {}'.format(LAMMPS_EXECUTABLE, i.name)
        proc = run(cmd, shell=True, cwd=i.parent , stdout=PIPE, stderr=PIPE)
    
    lammps_out_bulk = read_lammps_output(dir_path=input_path_bulk.parent)
    lammps_out_GB = read_lammps_output(dir_path=input_path_GB.parent)
    
    E_tot_bulk = lammps_out_bulk['final_energy'][-1]
    E_tot_GB = lammps_out_GB['final_energy'][-1]
    
    E_GB = (1 / (2 * bicrystal.boundary_area)) * (
        E_tot_GB - (bicrystal.num_atoms / bulk.num_atoms) * E_tot_bulk
    ) * UNIT_CONV
    print(f'{structure_code}: {E_GB}')
    E_GB_EAM.update({structure_code: E_GB})